In [1]:
# https://taxfoundation.org/data/all/state/state-income-tax-rates-2024/
import pandas as pd
tax_bracket_data = pd.read_excel("State-Individual-Income-Tax-Rates-and-Brackets-2015-2024_Tax_Foundation.xlsx", sheet_name=None)
state_dct ={
    "Ala.":"AL",
    "Alaska":"AK",
    "Ariz.":"AZ",
    "Ark.":"AR",
    "Calif.":"CA",
    "Colo.":"CO",
    "Conn.":"CT",
    "Del.":"DE",
    "Fla.":"FL",
    "Ga.":"GA",
    "Hawaii":"HI",
    "Idaho":"ID",
    "Ill.":"IL",
    "Ind.":"IN",
    "Iowa":"IA",
    "Kans.":"KS",
    "Ky.":"KY",
    "La.":"LA",
    "Maine":"ME",
    "Md.":"MD",
    "Mass.":"MA",
    "Mich.":"MI",
    "Minn.":"MN",
    "Miss.":"MS",
    "Mo.":"MO",
    "Mont.":"MT",
    "Nebr.":"NE",
    "Nev.":"NV",
    "N.H.":"NH",
    "N.J.":"NJ",
    "N.M.":"NM",
    "N.Y.":"NY",
    "N.C.":"NC",
    "N.D.":"ND",
    "Ohio":"OH",
    "Okla.":"OK",
    "Ore.":"OR",
    "Pa.":"PA",
    "R.I.":"RI",
    "S.C.":"SC",
    "S.D.":"SD",
    "Tenn.":"TN",
    "Texas":"TX",
    "Utah":"UT",
    "Vt.":"VT",
    "Va.":"VA",
    "Wash.":"WA",
    "W.Va.":"WV",
    "Wis.":"WI",
    "Wyo.":"WY",
    "D.C.":"DC"
    }

state_dct2 = {"Alabama":"AL",
              "Alaska":"AK", 
              "Arizona": "AZ",
                "Arkansas":"AR",
                "California":"CA",
                "Colorado":"CO",
                "Connecticut":"CT",
                "Delaware":"DE",
                "Florida":"FL",
                "Georgia":"GA",
                "Hawaii":"HI",
                "Idaho":"ID",
                "Illinois":"IL",
                "Indiana":"IN",
                "Iowa":"IA",
                "Kansas":"KS",
                "Kentucky":"KY",
                "Louisiana":"LA",
                "Maine":"ME",
                "Maryland":"MD",
                "Massachusetts":"MA",
                "Michigan":"MI",
                "Minnesota":"MN",
                "Mississippi":"MS",
                "Missouri":"MO",
                "Montana":"MT",
                "Nebraska":"NE",
                "Nevada":"NV",
                "New Hampshire":"NH",
                "New Jersey":"NJ",
                "New Mexico":"NM",
                "New York":"NY",
                "North Carolina":"NC",
                "North Dakota":"ND",
                "Ohio":"OH",
                "Oklahoma":"OK",
                "Oregon":"OR",
                "Pennsylvania":"PA",
                "Rhode Island":"RI",
                "South Carolina":"SC",
                "South Dakota":"SD",
                "Tennessee":"TN",
                "Texas":"TX",
                "Utah":"UT",
                "Vermont":"VT",
                "Virginia":"VA",
                "Washington":"WA",
                "West Virginia":"WV",
                "Wisconsin":"WI",
                "Wyoming":"WY",
                "District of Columbia":"DC"
                }


In [2]:
min_max_dct = {"Min":{},
               "Max":{}}
single_married = ["Single Filer","Married Filing Jointly"]
min_max_dct = {min_max: {sm: {year: {} for year in tax_bracket_data.keys()} for sm in single_married} for min_max in ("Min","Max")}


for year in tax_bracket_data.keys():
    tax_bracket_data[year] = tax_bracket_data[year].iloc[1:]
    tax_bracket_data[year]["State"] = tax_bracket_data[year]["Unnamed: 0"].map(state_dct).fillna(method='ffill')
    for sm in single_married:
        tax_bracket_data[year][sm] = pd.to_numeric(tax_bracket_data[year][sm], errors =  "coerce")
       
        for min_max in ("min","max"):
            min_max_data = tax_bracket_data[year][["State", sm]]
            min_max_dct[min_max.title()][sm][year] = getattr(min_max_data.groupby("State"), min_max)()[sm]


/tmp/ipykernel_47334/2502699505.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tax_bracket_data[year]["State"] = tax_bracket_data[year]["Unnamed: 0"].map(state_dct).fillna(method='ffill')
/tmp/ipykernel_47334/2502699505.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tax_bracket_data[year]["State"] = tax_bracket_data[year]["Unnamed: 0"].map(state_dct).fillna(method='ffill')
/tmp/ipykernel_47334/2502699505.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  tax_bracket_data[year]["State"] = tax_bracket_data[year]["Unnamed: 0"].map(state_dct).fillna(method='ffill')
/tmp/ipykernel_47334/2502699505.py:9: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.

In [4]:
min_max_dfs = {}
for sm in single_married:
    min_max_dfs[sm] = {}
    for min_max in ("min", "max"):
        min_max_dfs[sm][min_max] = pd.DataFrame(min_max_dct[min_max.title()][sm])
    min_max_dfs[sm] = pd.concat(min_max_dfs[sm].values(), keys = min_max_dfs[sm].keys()).reset_index()
    min_max_dfs[sm] = min_max_dfs[sm].rename(columns = {"level_0":"minmax"}).set_index(["State", "minmax"]).sort_index().sort_index(axis=1)
    min_max_dfs[sm] = min_max_dfs[sm].reset_index().melt(id_vars = ["State", "minmax"], var_name = "Year", value_name = "Income Tax Rate")
    


In [13]:
min_max_dfs[sm].set_index(["State", "Year"]).loc["MN"]
tax_bracket_data["2024"]

,Unnamed: 0,Single Filer,Unnamed: 2,Unnamed: 3,Married Filing Jointly,Unnamed: 5,Unnamed: 6,Standard Deduction,Unnamed: 8,Personal Exemption,Unnamed: 10,Unnamed: 11,State
1,Ala.,0.02,>,0,0.02,>,0,2500,7500,1500,3000,1000,AL
2,"(b, f, gg)",0.04,>,500,0.04,>,1000,NaN,NaN,NaN,NaN,NaN,AL
3,NaN,0.05,>,3000,0.05,>,6000,NaN,NaN,NaN,NaN,NaN,AL
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,AL
5,Alaska,NaN,NaN,NaN,NaN,NaN,NaN,n.a.,n.a.,n.a.,n.a.,n.a.,AK
...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,(ff) Rates include the additional tax at the r...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC
291,(gg) Local income taxes are excluded. Eleven s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC
292,(hh) State provides a state-defined personal e...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC
293,(ii) Standard deduction adjusted each year for...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DC


In [4]:
from homebrewedFunctions.functions import *
import plotly.express as px
regions_df = pd.read_csv("USCensusRegions.csv")
dfs = {}
# for min_max in ("Min", "Max"):
for sm in single_married:
    dfs[sm] = min_max_dfs[sm].pivot(index = ["State", "Year"], columns = "minmax", values = "Income Tax Rate").dropna().reset_index()
    dfs[sm]["Year"] = pd.to_datetime(dfs[sm]["Year"], format = "%Y")
    dfs[sm].sort_values(by = "Year", inplace = True)
    dfs[sm]["Year"] = dfs[sm]["Year"].dt.year.astype(str)
    dfs[sm].set_index(["State", "Year"], inplace = True)
    dfs[sm].sort_index()
figs = aggregated_line_dropdown(dfs, regions_df, f"TaxBrackets")
figs.write_html(f"outputs/State government finances/TaxBrackets.html")


# figs = {}
#     for state in df["State"].unique():
#         fig = px.line(df[df["State"]] == state],
#         x="Year", 
#         y="Income Tax Rate",
#         color="State",
#         facet_col="minmax")



In [93]:
import re
import numpy as np
def remove_parentheses_content(input_string):
    # This pattern removes the first occurrence of any content inside parentheses
    return re.sub(r'\(.*?\)', '', input_string)
IITR = pd.read_csv("2024 State Income Tax Rates and Brackets  Tax Foundation.csv")
IITR["State"] = IITR["State"].apply(remove_parentheses_content)
IITR["State"] = IITR["State"].str.strip()
IITR["State"] = IITR["State"].map(state_dct2)
IITR["index"] = np.arange(len(IITR))
IITR = IITR.set_index(["State", "index"]).rename(columns = {"Single Filer Rates": "Single", "Married Filing Jointly Rates": "Married"})[["Single", "Married"]]
for key, val in IITR.items():
    IITR[key] = val.str.replace("%", "").str.replace("none", "0")
    IITR[key] = pd.to_numeric(IITR[key], errors = "coerce").fillna(0)

min_IITR = IITR.reset_index().set_index(["State", "index"]).groupby("State").min()
max_IITR = IITR.reset_index().set_index(["State", "index"]).groupby("State").max()

# IITR = pd.DataFrame(max_CITR)#, columns = ["Corporate Income Tax Rate"]).reset_index()
# IITR=IITR.reset_index().sort_values(by="Rates", ascending = False)
# IITR

In [108]:
CITR = pd.read_csv("2024 State Corporate Income Tax Rates  Brackets.csv")
CITR["State"] = CITR["State"].map(state_dct2).str.strip()
CITR["Rates"] = CITR["Rates"].str.replace("%", "").astype(float).div(100)
min_CITR = CITR.groupby("State")["Rates"].min()
max_CITR = CITR.groupby("State")["Rates"].max()
# CITR = pd.concat([min_CITR, max_CITR], keys = ["Min", "Max"]).reset_index()
# CITR = CITR.rename(columns = {"level_0":"minmax"}).set_index(["State", "minmax"]).sort_index().sort_index(axis=1)
# CITR = CITR.reset_index().melt(id_vars = ["State", "minmax"], var_name = "Year", value_name = "Corporate Income Tax Rate")
CITR = pd.DataFrame(max_CITR)#, columns = ["Corporate Income Tax Rate"]).reset_index()
CITR=CITR.reset_index().sort_values(by="Rates", ascending = False)
CITR["Rates"] = CITR["Rates"].fillna(0).mul(100)
CITR.set_index("State", inplace = True)
CITR

,Rates
State,
MN,9.80
AK,9.40
ME,8.93
CA,8.84
VT,8.50
PA,8.49
MD,8.25
MA,8.00
WI,7.90


In [109]:
max_rates = max_IITR.copy()
max_rates["Corporate"] = CITR["Rates"]
max_rates.sort_values(by = "Corporate", ascending = False)

,Single,Married,Corporate
State,,,
MN,9.85,9.85,9.80
AK,0.00,0.00,9.40
ME,7.15,7.15,8.93
CA,13.30,13.30,8.84
VT,8.75,8.75,8.50
PA,3.07,3.07,8.49
MD,5.75,5.75,8.25
MA,9.00,9.00,8.00
WI,7.65,7.65,7.90


In [63]:
fig = px.bar(CITR, x = "State", y = "Rates")
fig.update_layout(title = "2024 State Corporate Income Tax Rates<br>(Maximum Rate for States with Non-Uniform Schedule)")
fig.write_html(f"outputs/State government finances/2024 State Corporate Income Tax Rates .html")

In [111]:


allCITR = pd.DataFrame(pd.concat([min_CITR,max_CITR], keys = ["Man", "Max"], axis = 0)).reset_index().rename(columns = {"Rates":"Corporate Income Tax Rate",
                                                                                             "level_0":"Measure"})
# allCITR=allCITR.sort_values(by=["Rates"], ascending = False)
allCITR.fillna(0, inplace = True)
allCITR = allCITR.reset_index().sort_values(by=["Corporate Income Tax Rate", "Measure"], ascending = False)
allCITR["Measure"][allCITR["Measure"] == "Man"] = "Min"
fig = px.bar(
    allCITR,
    x='State',
    y="Corporate Income Tax Rate",
    color='Measure',
    barmode="group",
    color_discrete_map={"Min":"blue", "Max":"red"},
    title='Clustered Bar Chart by State and Category')
fig.show()
fig.write_html("outputs/State government finances/2024MinMaxCITR.html")


/tmp/ipykernel_14654/2491761709.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

